# AppBuilder Assistant SDK FunctionCall

您可基于 Assistants SDK，可通过全代码形式创建和调试专属智能体，实现FunctionCall等功能。

以下通过一个端到端的示例，介绍如何使用Assistants SDK创建智能体，并调用AppBuilder SDK内置的工具组件。

该应用的场景是动物识别，通过Assistant SDK创建智能体，并调用AppBuilder SDK内置的动物识别组件，实现识别动物的功能。

Assistants API/SDK 正在内测中，敬请期待公测版本。

## 一、环境准备

首先需要安装AppBuilder-SDK代码库，若已在开发环境安装，则可跳过此步。

**注意：**: appbuilder-sdk 的python版本要求 3.9+，安装的SDK version >= 0.7.0

In [ ]:
!python3 -m pip install appbuilder-sdk

## 二、代码实现

### 2.1 创建Assitant & Thread & Message

我们此处调用 `appbuilder.AnimalRecognition()`组件，需要Assitant识别的图片为一只大熊猫，示例图如下：


<img src="https://bj.bcebos.com/v1/appbuilder/animal_recognize_test.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-01-24T12%3A19%3A16Z%2F-1%2Fhost%2F411bad53034fa8f9c6edbe5c4909d76ecf6fad6862cf937c03f8c5260d51c6ae" alt="drawing" width="1000"/>

In [ ]:
import os
import appbuilder

# 配置你的密钥，主要在这之前需要首先申请Assistant API的内测资格
os.environ["APPBUILDER_TOKEN"] = "your_appbuilder_token"

# 创建 Assitant
assistant = appbuilder.assistant.assistants.create(
    name="test_function",
    description="你是一个热心的朋友",
    instructions="请用友善的语气回答问题",
    tools=[
        {'type': 'function', 'function': appbuilder.AnimalRecognition().manifests[0]}
    ]
)

# 创建 Thread（即会话：conversation）
thread = appbuilder.assistant.threads.create()

image_url = "https://bj.bcebos.com/v1/appbuilder/animal_recognize_test.png?" \
            "authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-01-24T" \
            "12%3A19%3A16Z%2F-1%2Fhost%2F411bad53034fa8f9c6edbe5c4909d76ecf6fad68" \
            "62cf937c03f8c5260d51c6ae"

origin_query = "我有一张图片，url是: {}, 麻烦帮我看看这是什么动物".format(image_url)

# 在Thread中添加一条Message，内容为原始的query
appbuilder.assistant.threads.messages.create(
    thread_id=thread.id,
    content=origin_query,
)

### 2.2 第一次运行，触发函数调用

我们向Assistant询问 `我有一张图片，url是: X, 麻烦帮我看看这是什么动物`, Assistant会调用我们预先配置好的函数，并返回调用参数，我们收到调用触发的信号后，在本地调用函数

In [ ]:
# 触发第一次run，并收到函数本地调用的信号
run_result = appbuilder.assistant.threads.runs.run(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

print("\nFirst run result: {}\n".format(run_result.model_dump_json(indent=4)))

运行结果为：

```bash
First run result: {
    "id": "run_089cffe9a82944bd9cdb87a8a0d8b54f",
    "object": "run.result",
    "assistant_id": "asst_922cacd503354349a1a0a5afb0be0408",
    "thread_id": "thread_523a4aacc6a845a0b8252009af5771ea",
    "model": "",
    "instructions": "",
    "thought_instructions": "",
    "chat_instructions": "",
    "tools": null,
    "file_ids": null,
    "status": "requires_action",
    "required_action": {
        "type": "submit_tool_outputs",
        "submit_tool_outputs": {
            "tool_calls": [
                {
                    "id": "call-thread_523a4aacc6a845a0b8252009af5771ea-run_089cffe9a82944bd9cdb87a8a0d8b54f-step-1",
                    "type": "function",
                    "function": {
                        "name": "animal_rec",
                        "arguments": "{\"img_url\":\"https://bj.bcebos.com/v1/appbuilder/animal_recognize_test.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-01-24T12%3A19%3A16Z%2F-1%2Fhost%2F411bad53034fa8f9c6edbe5c4909d76ecf6fad6862cf937c03f8c5260d51c6ae\"}",
                        "output": ""
                    }
                }
            ]
        }
    },
    "last_error": null,
    "final_answer": null,
    "created_at": 1713868218144,
    "started_at": 1713868218164,
    "expired_at": 0,
    "cancelled_at": 0,
    "failed_at": 0,
    "completed_at": 0
}
```

### 2.3 识别函数调用的触发信号，调用本地函数

In [ ]:
# 触发函数调用的标志是： status == 'requires_action'
assert run_result.status == 'requires_action'

# 获取函数调用的信息
tool_call = run_result.required_action.submit_tool_outputs.tool_calls[0]

# 调用工具组件的 tool_eval 方法，获取函数本地的运行结果
func_res = appbuilder.AnimalRecognition().tool_eval(
        name="animal_rec",
        streaming=True,
        origin_query=origin_query,
        **eval(tool_call.function.arguments))

func_message = ""
for res in func_res:
    func_message += res
print("\nFunction result: {}\n".format(func_message))

运行结果为：


```bash
Function result: 模型识别结果为：
类别: 国宝大熊猫 置信度: 0.975161
```

### 2.4 上传函数运行结果，继续触发运行，得到完整对话结果

In [ ]:
run_result = appbuilder.assistant.threads.runs.run(
    thread_id=thread.id,
    assistant_id=assistant.id,
    tool_output={
        "tool_call_id":tool_call.id,
        "output": func_message,
        "run_id": run_result.id
    },
)
print("\nFinal run result: {}\n".format(run_result.model_dump_json(indent=4)))

运行结果为：

```bash

Final run result: {
    "id": "run_089cffe9a82944bd9cdb87a8a0d8b54f",
    "object": "run.result",
    "assistant_id": "asst_922cacd503354349a1a0a5afb0be0408",
    "thread_id": "thread_523a4aacc6a845a0b8252009af5771ea",
    "model": "",
    "instructions": "",
    "thought_instructions": "",
    "chat_instructions": "",
    "tools": null,
    "file_ids": null,
    "status": "completed",
    "required_action": null,
    "last_error": null,
    "final_answer": {
        "type": "message",
        "message": {
            "message_id": "chatmsg_8d54f8c3c6714665baeda5eed013a5e9",
            "content": {
                "type": "text",
                "text": {
                    "value": "根据您的图片，我识别出图片中的动物是**国宝大熊猫**，识别结果的置信度很高，为0.975161。如果您还有其他问题或需要识别其他图片，请随时告诉我。",
                    "annotations": null
                }
            }
        }
    },
    "created_at": 1713868218144,
    "started_at": 1713868239553,
    "expired_at": 0,
    "cancelled_at": 0,
    "failed_at": 0,
    "completed_at": 1713868251090
}
```

## 总结

在本示例中，展示了如何使用Assistant SDK，并调用AppBuilder中的能力组件，原子化的进行Agent流程编排与交互